In [1]:
import subprocess
import sys

In [2]:
import harmonypy as hm
import pandas as pd
import numpy as np
import os
from pycytominer.cyto_utils import infer_cp_features
from sklearn.decomposition import IncrementalPCA

In [3]:
! pip install harmonypy

### The line below adds path flexibility.

In [4]:
norm_feat_files_list = []
datadir = "/home/ubuntu/efs/2018_05_30_ResistanceMechanisms_Kapoor/workspace/software/cell-painting-batch-effects/0.process-data/data"
for subdir, dirs, files in os.walk(datadir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if ".csv" in filepath:
            norm_feat_files_list.append(filepath)

In [5]:
cell_data = pd.read_csv(norm_feat_files_list[0])
for file in norm_feat_files_list[1:]:
    cell_data = pd.concat([cell_data, pd.read_csv(file)])
cell_data = cell_data.dropna(axis=1)

In [6]:
cell_data = cell_data.reset_index(drop=True)

In [7]:
cell_metadata = infer_cp_features(cell_data, metadata=True)

In [8]:
cell_pca_doer = IncrementalPCA(n_components=50)
cell_pca_doer = cell_pca_doer.fit(cell_data.drop(cell_metadata, axis=1).sample(35000))

In [9]:
cells_data_features = cell_data.drop(cell_metadata, axis=1)
cells_data_features

,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,Cells_AreaShape_Solidity,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,...,Nuclei_Texture_SumEntropy_ER_5_00,Nuclei_Texture_SumEntropy_ER_5_01,Nuclei_Texture_SumEntropy_ER_5_02,Nuclei_Texture_SumEntropy_ER_5_03,Nuclei_Texture_SumEntropy_RNA_10_01,Nuclei_Texture_SumEntropy_RNA_10_03,Nuclei_Texture_SumEntropy_RNA_5_00,Nuclei_Texture_SumEntropy_RNA_5_01,Nuclei_Texture_SumEntropy_RNA_5_02,Nuclei_Texture_SumEntropy_RNA_5_03
0,-2.623100,2.734000,1.22040,1.613300,1.21090,1.58270,0.22113,0.88020,-1.867000,-1.42450,...,0.021625,0.021247,-0.089120,-0.10300,0.630760,0.026159,0.401070,0.432290,0.326670,0.355280
1,-0.027616,0.179800,1.10550,-0.859210,0.60705,0.53513,-1.03650,1.37940,0.332700,-1.27860,...,-1.203900,-1.140500,-0.934600,-0.95611,-1.849400,-1.616000,-1.768000,-1.630700,-1.609200,-1.521500
2,-0.902760,1.334800,2.22570,1.426200,1.79280,1.25340,1.01600,0.51280,-0.359260,-0.36884,...,1.489200,1.618600,1.566700,1.31610,1.738700,1.810800,1.922700,1.805500,1.946100,1.976800
3,-0.036605,-0.032211,-0.33675,1.295000,-0.43261,0.28043,0.10611,0.15768,0.527050,-0.64978,...,-0.062876,0.070932,-0.023483,-0.13277,0.224620,-0.314420,0.206430,0.375760,0.354910,0.236770
4,0.936130,0.434810,0.41605,1.377100,0.79573,-0.77460,0.63395,-0.14980,-0.097622,1.66930,...,0.742480,0.708070,0.784930,0.81048,0.602590,0.607620,0.453030,0.470750,0.499020,0.491030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.674820,-1.320300,-0.36300,-0.273800,-0.89684,-0.78098,-0.89469,0.32294,0.104960,0.13891,...,0.319060,0.201500,0.423360,0.69212,0.021289,0.537550,-0.037876,-0.170700,0.093962,0.208250
39996,0.557890,1.117700,1.11740,-1.612400,0.67644,0.19431,0.71514,-0.32033,1.366700,0.19262,...,-0.421860,-0.354100,-0.376450,-0.63222,-0.467860,-0.525030,-0.775850,-0.648640,-0.655950,-0.862660
39997,-1.163300,1.315700,0.73353,0.952560,1.04380,1.17800,0.75552,0.79994,-0.316780,-0.76894,...,0.404330,0.402850,0.358940,0.38188,0.199270,0.585180,0.257150,0.190870,0.312780,0.361420
39998,1.622000,-1.237700,-1.98110,0.013972,-2.44300,-2.52550,-1.62540,-2.13700,-0.353920,-0.82954,...,0.173500,0.168260,0.056677,0.18035,0.007535,0.117770,0.099087,0.040319,0.075434,0.007325


In [10]:
cell_data_pcs = cell_pca_doer.transform(cells_data_features)

In [11]:
cell_data_pcs = pd.DataFrame(cell_data_pcs).add_prefix("pca_")

In [12]:
# Random-seed 0 used within.
harmony_out = hm.run_harmony(cell_data_pcs, cell_data.loc[:, cell_metadata], ["Metadata_Plate", "Metadata_Well"])

2020-07-30 14:11:38,388 - harmonypy - INFO - Iteration 1 of 10
2020-07-30 14:11:46,099 - harmonypy - INFO - Iteration 2 of 10
2020-07-30 14:11:53,817 - harmonypy - INFO - Iteration 3 of 10
2020-07-30 14:12:01,553 - harmonypy - INFO - Converged after 3 iterations


In [13]:
harmony_cell_data = pd.DataFrame(np.transpose(harmony_out.Z_corr)).add_prefix("pca_").add_suffix("_adjusted")

In [14]:
harmony_cell_data = pd.concat([cell_data.loc[:, cell_metadata], harmony_cell_data], axis=1)

In [15]:
harmony_cell_data.to_csv("Corrected_Cell_Data_All_Batches.csv.gz", compression='gzip')

In [16]:
cell_data.to_csv("Uncorrected_Cell_Data_All_Batches.csv.gz", compression='gzip')